Importing the 20NewsGroups dataset consisting of 11314 articles in the training dataset and 7532  articles in the test dataset accross 20 classes.

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
train = fetch_20newsgroups(subset='train', shuffle=True)
test = fetch_20newsgroups(subset='test', shuffle=True)

Next we Vectorize the articles in the Corpus.
For this we use sci-kit learn's CountVectorizer to create a sparse matrix of the count of each word in an article
For better results we then calculate the inverse term frequency for the words using sci-kit learn's TfidfTransformer

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.data)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print (X_train_tfidf[0:4,0:20000])


  (0, 4605)	0.06332603952480324
  (0, 16574)	0.14155752531572685
  (0, 18299)	0.138749083899155
  (1, 7797)	0.13724375024886204
  (1, 2927)	0.05212944077716299
  (2, 12197)	0.05168179280403425
  (2, 15032)	0.07834044496813063
  (2, 6449)	0.0681281384860916
  (2, 5023)	0.13698619641739623
  (2, 5811)	0.2878251559842457
  (2, 6028)	0.10554465088856506
  (2, 3412)	0.0622873125208309
  (3, 4155)	0.05353413616615429
  (3, 18618)	0.14195950717692907


Having got the sparse matrix, we would now apply classification Algorithms on this vectorized word matrix to predic classes for data in test dataset.
Starting with K-Nearest Neighbours


In [5]:
from sklearn.pipeline import Pipeline
from sklearn import neighbors
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', neighbors.KNeighborsClassifier())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
np.mean(predicted == test.target)

0.6591874668082847

Now we apply Support Vector Machine algorithm


In [38]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings("ignore")
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
np.mean(predicted == test.target)

0.8516994158258099

Now we apply Naive Bayes

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
np.mean(predicted == test.target)


0.7738980350504514

All this while we used Bag-Of-Words technique to vectorize the dataset.
Here we apply ngrams technique to create the sparse matrix. Let's have an example as how n-grams is differnt from Bag-of-Words and what it actually does.

In [54]:
ngram_vectorizer = CountVectorizer()
counts = ngram_vectorizer.fit_transform(['Anagh', 'Anmol'])
print("Bag-of-Words")
print(ngram_vectorizer.get_feature_names())
print(counts.toarray().astype(int))

ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))
counts = ngram_vectorizer.fit_transform(['Anagh', 'Anmol'])
print("Bi-grams")
print(ngram_vectorizer.get_feature_names())
print(counts.toarray().astype(int))

ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(3, 3))
counts = ngram_vectorizer.fit_transform(['Anagh', 'Anmol'])
print("Tri-grams")
print(ngram_vectorizer.get_feature_names())
print(counts.toarray().astype(int))


Bag-of-Words
['anagh', 'anmol']
[[1 0]
 [0 1]]
Bi-grams
[' a', 'ag', 'an', 'gh', 'h ', 'l ', 'mo', 'na', 'nm', 'ol']
[[1 1 1 1 1 0 0 1 0 0]
 [1 0 1 0 0 1 1 0 1 1]]
Tri-grams
[' an', 'agh', 'ana', 'anm', 'gh ', 'mol', 'nag', 'nmo', 'ol ']
[[1 1 1 0 1 0 1 0 0]
 [1 0 0 1 0 1 0 1 1]]


What if we need to apply Bag-of-2grams, or in other words club two consecutive words in a document, then vectorize

In [60]:
ngram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
counts = ngram_vectorizer.fit_transform(['Today it is Allahabad', 'Tomorrow it will be Prayagraj'])
print("Bag-of-2grams")
print(ngram_vectorizer.get_feature_names())
print(counts.toarray().astype(int))

Bag-of-2grams
['be prayagraj', 'is allahabad', 'it is', 'it will', 'today it', 'tomorrow it', 'will be']
[[0 1 1 0 1 0 0]
 [1 0 0 1 0 1 1]]


Now we will apply n-grams on our dataset and will then apply SVM classification algorithm. We will compare the accuracies for uni-gram, bi-gram and tri-gram vectorization on character level.

In [46]:
text_clf = Pipeline([('vect', CountVectorizer(analyzer='char_wb', ngram_range=(1, 1))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
print("For uni-grams : ",np.mean(predicted == test.target))

text_clf = Pipeline([('vect', CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
print("For bi-grams : ",np.mean(predicted == test.target))

text_clf = Pipeline([('vect', CountVectorizer(analyzer='char_wb', ngram_range=(3, 3))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
print("For tri-grams : ",np.mean(predicted == test.target))


For uni-grams :  0.15493892724375996
For bi-grams :  0.6437865108868827
For tri-grams :  0.806558682952735


Now applying Bag-of-1gram(Same as bag of words), Bag-of-2grams and Bag-of-3grams to our dataset which by the way is out actual intention.

In [64]:
text_clf = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(1, 1))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
print("For Bag-of-1-grams : ",np.mean(predicted == test.target))

text_clf = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(2, 2))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
print("For Bag-of-2-grams : ",np.mean(predicted == test.target))

text_clf = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(3, 3))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())])
text_clf.fit(train.data, train.target)
predicted = text_clf.predict(test.data)
print("For Bag-of-3-grams : ",np.mean(predicted == test.target))

For Bag-of-1-grams :  0.8499734466277217
For Bag-of-2-grams :  0.8021773765268189
For Bag-of-3-grams :  0.7169410515135423
